In [1]:
from parser import DNS_Shop_Parser
from keybert import KeyBERT
from pymystem3 import Mystem
from wordcloud import WordCloud
import  multiprocessing as mp
import nltk
import re

In [2]:
# TODO многпоточная лемматизация

In [3]:
# nltk.download('punkt')
# nltk.download('stopwords')
# nltk.download('punkt_tab')
# stopwords = nltk.corpus.stopwords.words('russian')

На вход приходит список словарей. Каждый словарь может содержать ключи 'Достоинства', 'Недостатки', 'Комментарий', 'Фото', а также ключи вида "Дополнение от 27 мая 2025 года", от которых необходимо избавиться.\
Наблюдение: если в отзыве есть "Дополнение...", то есть как минимум 'Достоинства' или 'Недостатки' или 'Комментарий'.

In [4]:
def convert_to_list_of_reviews(reviews):
    dict_of_none = {'Достоинства' : None, 'Недостатки' : None, 'Комментарий' : None, 'Фото' : None}
    texts = []
    for review in reviews:
        if review != dict_of_none:
            texts.append('') # if all the values in review is None, but if it has some key, that is not in dict_of_none -> memory leak. need to fix that
            for key, value in review.items():
                if (value is not None) and (key in dict_of_none.keys()):
                    texts[-1] += ' ' + value + ' '
    return texts

Препроцессинг текста

In [5]:
def preprocess_list_of_texts(list_of_texts) -> list:
    preprocessed_list_of_texts = []
    for text in list_of_texts:
        preprocessed_list_of_texts.append(text.lower())
        # texts[i] = re.sub(r'[^\w\s]', '', texts[i].lower())
    return preprocessed_list_of_texts

Возможность пердварительной лемматизации отзывов, перед извлечением ключевых N-грамм

In [6]:
def get_lemmatized_text(list_of_texts, mystem: Mystem) -> list:
    lemmatized_texts = []
    for text in list_of_texts:
        lemmatized_texts.append(''.join(mystem.lemmatize(text)))
    return lemmatized_texts

Извлечение ключевых N-грамм из каждого отзыва

In [7]:
def extract_keyphrases(corpus_list, model : KeyBERT, top_n=2, keyphrase_ngram_range=(1,2)) -> list:
    raw_keywords_list = []
    for corpus in corpus_list:
        raw_keywords_list.append(model.extract_keywords(docs=corpus, keyphrase_ngram_range=keyphrase_ngram_range,
                                                    stop_words=None, top_n=top_n, diversity=0.7))
    return raw_keywords_list

Функция необходима для преобразования списка, получаемого после **extract_keyphrases()**, в словарь, пригодный для исопльзования в библиотеке wordcloud

In [8]:
def get_dict_keywords(keywords_lists, threshold=0.5) -> dict:
    '''destined for wordcloud.
       Returns dictionary: key = keyword, value = [0,1]'''
    propper_keywords = {}
    for keyword_list in keywords_lists:
        for keyword in keyword_list:
            if keyword[1] >= threshold:
                propper_keywords[keyword[0]] = keyword[1]
    return propper_keywords

In [ ]:
parser = DNS_Shop_Parser()

Извлечение отзывов из текстового файла на время тестов, воизбежание бана в ДНС

In [10]:
reviews = []
with open('raw_html_big.txt', 'r', encoding='utf-8') as file:
    reviews = parser.extract_reviews(file.read()) # propper form: list of dictionaries

In [11]:
# Извлечение отзывов задуманным способом
# parser.open_DNS_site() # execute once
# reviews = parser.get_product_reviews() # propper form: list of dictionaries

Препроцессинг корпуса

In [12]:
preprocessed_list = preprocess_list_of_texts(convert_to_list_of_reviews(reviews))

Токенизация и очистка от стоп-слов (не используется, поскольку отсутствие стоп-слов портит семантику корпуса => несвязанные ключевые N-граммы)

In [13]:
# list_tokens = []
# for text in raw_list_corpus:
#     list_tokens.append(nltk.tokenize.word_tokenize(text, language='russian'))

# cleaned_corpus_list = []
# allowed_stopwords = ['не', 'ни']
# for review in list_tokens:
#     cleaned_corpus = []
#     for token in review:
#         if (token not in stopwords) or (token in allowed_stopwords):
#             cleaned_corpus.append(token)
#     cleaned_corpus_list.append(' '.join(cleaned_corpus))

In [14]:
model = KeyBERT('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

Облако слов без лемматизации крпуса

In [15]:
raw_keywords_list = extract_keyphrases(preprocessed_list, model, 2, (1,2))

In [16]:
raw_keywords_list

[[('система виснет', 0.4924), ('диски нет', 0.4436)],
 [('месяц пользуюсь', 0.7255), ('уже месяц', 0.6457)],
 [('деградация накопителя', 0.6685), ('началась деградация', 0.5528)],
 [('работает быстро', 0.7167), ('стабильно работает', 0.6435)],
 [('цена не', 0.6774), ('цена', 0.6176)],
 [('всё работает', 0.9927), ('работает', 0.8868)],
 [('скорость передачи', 0.6561), ('скорость', 0.6001)],
 [('летают игры', 0.6476), ('игры теперь', 0.5506)],
 [('чудо китайской', 0.6931), ('китайской инженерии', 0.662)],
 [('цене 512гб', 0.6759), ('512гб обманывают', 0.6745)],
 [('такой диск', 0.5786), ('цена хз', 0.5)],
 [('скорости цена', 0.7007), ('заявленные скорости', 0.6424)],
 [('обычном hdd', 0.4446), ('hdd', 0.3993)],
 [('минут чтения', 0.5387), ('загрузки скорость', 0.5287)],
 [('диски бюджетного', 0.5479), ('data разных', 0.5289)],
 [('без приключений', 0.8054), ('работает без', 0.7247)],
 [('так нету', 0.6403), ('мне', 0.637)],
 [('быстрый безотказный', 0.7252), ('поставил пс3', 0.6675)],
 [

In [17]:
keywords_dict = get_dict_keywords(raw_keywords_list)

In [18]:
wordcloud = WordCloud(width = 1000, 
                      height = 1000, 
                      background_color='black', 
                      margin=20, 
                      colormap='Pastel1', 
                      collocations=True, 
                      random_state=42)

wordcloud.generate_from_frequencies(keywords_dict)
image = wordcloud.to_image()
image.show()
image.save('no_lemmatized_before_keyphrase_extraction.PNG')

Облако слов с лемматизацией корпуса

In [19]:
mystem = Mystem()

In [20]:
# При таком подходе лемматизация занимает чуть более 7 минут
# lemmatized_list = get_lemmatized_text(preprocessed_list, mystem)    # Необходима оптимизация + многопоточность. 
                                                                    # Другой вариант - сконкатенировать список в один большой текст и добавить разделитель. Далее по разделителю преобразовать в список

In [21]:
splitter = '!@#$%^&*()' # Костыльный разделитель.
lemmatized_text = ''.join(mystem.lemmatize(splitter.join(preprocessed_list)))
lemmatized_list = lemmatized_text.split(splitter)

In [22]:
raw_keywords_list = extract_keyphrases(lemmatized_list, model, 2, (1,2))
keywords_dict = get_dict_keywords(raw_keywords_list)

In [23]:
wordcloud = WordCloud(width = 1000, 
                      height = 1000, 
                      background_color='black', 
                      margin=20, 
                      colormap='Pastel1', 
                      collocations=True, 
                      random_state=42)

wordcloud.generate_from_frequencies(keywords_dict)
image = wordcloud.to_image()
image.show()
image.save('lemmatized_before_keyphrase_extraction.PNG')